In [ ]:
import numpy as np
import timeit

import matplotlib.pyplot as plt

import torch
import torch.optim as optim

import sys
sys.path.insert(0, './../')
import sdss_psf
import simulated_datasets_lib
import starnet_vae_lib

import inv_KL_objective_lib as objectives_lib

import image_utils

import time

import json

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('torch version: ', torch.__version__)

from copy import deepcopy

In [ ]:
# load PSF
psf_fit_file = '../../celeste_net/sdss_stage_dir/2583/2/136/psField-002583-2-0136.fit'
print('psf file: \n', psf_fit_file)

In [ ]:
np.random.seed(22)
_ = torch.manual_seed(22)

# Draw data

In [ ]:
# data parameters
with open('../data/default_star_parameters.json', 'r') as fp:
    data_params = json.load(fp)

data_params['slen'] = 101
data_params['min_stars'] = 2000
data_params['max_stars'] = 2000
data_params['alpha'] = 0.5


In [ ]:
n_images = 1

simulated_dataset = \
    simulated_datasets_lib.load_dataset_from_params(psf_fit_file,
                            data_params,
                            n_images = n_images,
                            add_noise = True)


In [ ]:
# true parameters
batchsize = 1

loader = torch.utils.data.DataLoader(
                 dataset=simulated_dataset,
                 batch_size=batchsize,
                 shuffle=False)

for _, data in enumerate(loader):
    true_full_fluxes = data['fluxes']
    true_full_locs = data['locs']
    images_full = data['image']
    
    break


In [ ]:
# histogram of fluxes
plt.hist(np.log10(true_full_fluxes.numpy().flatten()))

In [ ]:
plt.matshow(images_full.squeeze());

In [ ]:
simulated_dataset.locs.numpy()

# Load VAE

In [ ]:
star_encoder = starnet_vae_lib.StarEncoder(full_slen = data_params['slen'],
                                            stamp_slen = 9,
                                            step = 4,
                                            edge_padding = 2, 
                                            n_bands = 1,
                                            max_detections = 8)

In [ ]:
star_encoder.load_state_dict(torch.load('../fits/starnet_invKL_encoder_batched_images_2000stars_smallpatch3', 
                               map_location=lambda storage, loc: storage))
star_encoder.eval(); 

In [ ]:
objectives_lib.eval_star_encoder_loss(star_encoder, loader, train = False)

In [ ]:
losses = np.loadtxt('../fits/test_losses_2000stars_smallpatch2')

In [ ]:
n_losses = losses.shape[1]
plt.plot(np.arange(n_losses - 1), losses[0, 1:n_losses])
plt.scatter(np.arange(n_losses - 1), losses[0, 1:n_losses], marker = 'x')

losses2 = np.loadtxt('../fits/test_losses_2000stars_smallpatch3')
n_losses = losses2.shape[1]
plt.plot(np.arange(n_losses - 1), losses2[0, 1:n_losses])
plt.scatter(np.arange(n_losses - 1), losses2[0, 1:n_losses], marker = 'x')


# Get inferred parameters

In [ ]:
image_stamps, true_subimage_locs, true_subimage_fluxes, true_n_stars, is_on_array = \
        star_encoder.get_image_stamps(images_full, true_full_locs, true_full_fluxes, 
                                      trim_images = False)

In [ ]:
plt.hist(true_n_stars, bins=np.arange(max(true_n_stars) + 1));

In [ ]:
_backgrounds = torch.ones((image_stamps.shape[0], 1, 1, 1)) * data_params['sky_intensity']

In [ ]:
loss, counter_loss, locs_loss, fluxes_loss, perm = \
    objectives_lib.get_encoder_loss(star_encoder, images_full, _backgrounds, true_full_locs, true_full_fluxes)

In [ ]:
plt.hist(counter_loss.detach())

In [ ]:
# if more than max detections ...
_true_n_stars = true_n_stars.clone()
_true_n_stars[_true_n_stars > star_encoder.max_detections] = star_encoder.max_detections
    
logit_loc_mean, logit_loc_log_var, \
    log_flux_mean, log_flux_log_var, log_probs = \
        star_encoder(image_stamps, data_params['sky_intensity'], _true_n_stars)

# check parameters

In [ ]:
# permute true parameters 
def permute_params(locs, fluxes, perm): 
    batchsize = perm.shape[0]
    max_stars = perm.shape[1]

    locs_perm = torch.zeros((batchsize, max_stars, 2))
    fluxes_perm = torch.zeros((batchsize, max_stars))
    seq_tensor = torch.LongTensor([i for i in range(batchsize)])

    for i in range(max_stars):
        locs_perm[:, i, :] = locs[seq_tensor, perm[:, i], :]
        fluxes_perm[:, i] = fluxes[seq_tensor, perm[:, i]]
        
    return locs_perm, fluxes_perm

In [ ]:
locs_perm, fluxes_perm = permute_params(logit_loc_mean, log_flux_mean, perm)

In [ ]:
map_locs = torch.sigmoid(locs_perm) * is_on_array.unsqueeze(2).float()
map_fluxes = fluxes_perm * is_on_array.float()

In [ ]:
plt.plot(map_locs.flatten()[map_locs.flatten() > 0].detach(), 
         true_subimage_locs.flatten()[true_subimage_locs.flatten() > 0], '+')

plt.plot(map_locs.flatten()[map_locs.flatten() > 0].detach(), 
         map_locs.flatten()[map_locs.flatten() > 0].detach(), '-')

plt.xlabel('estimated')
plt.ylabel('truth')

In [ ]:
plt.plot(map_fluxes.flatten()[map_fluxes.flatten() > 0].detach(), 
         torch.log(true_subimage_fluxes.flatten()[true_subimage_fluxes.flatten() > 0]), '+')

plt.plot(map_fluxes.flatten()[map_fluxes.flatten() > 0].detach(), 
         map_fluxes.flatten()[map_fluxes.flatten() > 0].detach(), '-')

plt.xlabel('estimated')
plt.ylabel('truth')

# Check reconstructions 

In [ ]:
# get map estimates for image patches
map_n_stars = torch.argmax(log_probs, dim = 1)
is_on_array = objectives_lib.get_is_on_from_n_stars(map_n_stars, star_encoder.max_detections)

map_locs = torch.sigmoid(logit_loc_mean).detach() # * is_on_array.unsqueeze(2).float()
map_fluxes = torch.exp(log_flux_mean).detach() * is_on_array.float()

In [ ]:
# convert patch parameters to parameters on the full image 
map_locs_full_image, map_fluxes_full_image, n_stars = \
    image_utils.get_full_params_from_patch_params(map_locs, 
                                                  map_fluxes,
                                                    is_on_array,
                                                    star_encoder.tile_coords,
                                                    star_encoder.full_slen,
                                                    star_encoder.stamp_slen,
                                                    star_encoder.edge_padding,
                                                    star_encoder.batchsize)

In [ ]:
# get reconsstructed mean
vae_recon_mean = simulated_dataset.simulator.draw_image_from_params(locs = map_locs_full_image, 
                                                                    fluxes = map_fluxes_full_image,
                                                                     n_stars = n_stars, 
                                                                     add_noise = False).squeeze()

In [ ]:
fig, axarr = plt.subplots(1, 3, figsize=(15, 4))
im0 = axarr[0].matshow(images_full.squeeze())
fig.colorbar(im0, ax = axarr[0])

im1 = axarr[1].matshow(vae_recon_mean.squeeze())
fig.colorbar(im1, ax = axarr[1])

im2 = axarr[2].matshow(vae_recon_mean.squeeze() - images_full.squeeze())
fig.colorbar(im2, ax = axarr[2])

# check image patches

In [ ]:
import plotting_utils

In [ ]:
for i in range(1): 
    f, axarr = plt.subplots(1, 3, figsize=(16, 6))
    indx = 206 # int(np.random.choice(image_stamps.shape[0], 1))
    
    x0 = int(star_encoder.tile_coords[indx, 0])
    x1 = int(star_encoder.tile_coords[indx, 1]) 
    
    # plot image stamp
    im0 = axarr[0].matshow(image_stamps[indx].squeeze())
    im0 = fig.colorbar(im0, ax=axarr[0])
    
    # plot true locations      
    patch_slen = (star_encoder.stamp_slen - 2 * star_encoder.edge_padding)
    axarr[0].scatter(true_subimage_locs[indx, 0:true_n_stars[indx], 1] * (patch_slen - 1) + \
                         star_encoder.edge_padding, 
                    true_subimage_locs[indx, 0:true_n_stars[indx], 0] * (patch_slen - 1) + \
                         star_encoder.edge_padding, 
                    color = 'b')
    
    axarr[0].scatter(map_locs[indx, 0:map_n_stars[indx], 1] * (patch_slen - 1) + star_encoder.edge_padding, 
                    map_locs[indx, 0:map_n_stars[indx], 0] * (patch_slen - 1) + star_encoder.edge_padding, 
                    color = 'r', marker = 'x')
    
    
    axarr[0].axvline(x=2, color = 'r')
    axarr[0].axvline(x=6, color = 'r')
    axarr[0].axhline(y=2, color = 'r')
    axarr[0].axhline(y=6, color = 'r')
    
    axarr[0].set_title('observed; coords {}\n'.format([x0, x1]))
    
    # plot reconstruction
    recon_patch = vae_recon_mean[x0:(x0+star_encoder.stamp_slen), 
                                   x1:(x1+star_encoder.stamp_slen)]
    im1 = axarr[1].matshow(recon_patch)
    
    axarr[1].axvline(x=2, color = 'r')
    axarr[1].axvline(x=6, color = 'r')
    axarr[1].axhline(y=2, color = 'r')
    axarr[1].axhline(y=6, color = 'r')
    
    axarr[1].scatter(map_locs[indx, :, 1] * (patch_slen - 1) + star_encoder.edge_padding, 
                    map_locs[indx, :, 0] * (patch_slen - 1) + star_encoder.edge_padding, 
                    color = 'r', marker = 'x')
    fig.colorbar(im1, ax=axarr[1])
    
    
    # plot residual
    im2 = axarr[2].matshow((recon_patch - image_stamps[indx].squeeze()))
    fig.colorbar(im2, ax=axarr[2])

In [ ]:
torch.where((star_encoder.tile_coords[:, 0] == 28) & (star_encoder.tile_coords[:, 1] == 56))

In [ ]:
f, axarr = plt.subplots(1, 3, figsize=(16, 6))
indx = 182 # int(np.random.choice(image_stamps.shape[0], 1))

x0 = int(star_encoder.tile_coords[indx, 0])
x1 = int(star_encoder.tile_coords[indx, 1]) 

# plot image stamp
im0 = axarr[0].matshow(image_stamps[indx].squeeze())
im0 = fig.colorbar(im0, ax=axarr[0])

# plot true locations      
patch_slen = (star_encoder.stamp_slen - 2 * star_encoder.edge_padding)
axarr[0].scatter(true_subimage_locs[indx, 0:true_n_stars[indx], 1] * (patch_slen - 1) + \
                     star_encoder.edge_padding, 
                true_subimage_locs[indx, 0:true_n_stars[indx], 0] * (patch_slen - 1) + \
                     star_encoder.edge_padding, 
                color = 'b')

axarr[0].scatter(map_locs[indx, 0:map_n_stars[indx], 1] * (patch_slen - 1) + star_encoder.edge_padding, 
                map_locs[indx, 0:map_n_stars[indx], 0] * (patch_slen - 1) + star_encoder.edge_padding, 
                color = 'r', marker = 'x')


axarr[0].axvline(x=2, color = 'r')
axarr[0].axvline(x=6, color = 'r')
axarr[0].axhline(y=2, color = 'r')
axarr[0].axhline(y=6, color = 'r')

axarr[0].set_title('observed; coords {}\n'.format([x0, x1]))

# plot reconstruction
recon_patch = vae_recon_mean[x0:(x0+star_encoder.stamp_slen), 
                               x1:(x1+star_encoder.stamp_slen)]
im1 = axarr[1].matshow(recon_patch)

axarr[1].axvline(x=2, color = 'r')
axarr[1].axvline(x=6, color = 'r')
axarr[1].axhline(y=2, color = 'r')
axarr[1].axhline(y=6, color = 'r')

axarr[1].scatter(map_locs[indx, :, 1] * (patch_slen - 1) + star_encoder.edge_padding, 
                map_locs[indx, :, 0] * (patch_slen - 1) + star_encoder.edge_padding, 
                color = 'r', marker = 'x')
fig.colorbar(im1, ax=axarr[1])


# plot residual
im2 = axarr[2].matshow((recon_patch - image_stamps[indx].squeeze()))
fig.colorbar(im2, ax=axarr[2])

In [ ]:
f, axarr = plt.subplots(1, 3, figsize=(16, 6))

plotting_utils.plot_subimage(axarr[0], images_full.squeeze(),
                            map_locs_full_image.squeeze() * (images_full.shape[-1] - 1), 
                            true_full_locs.squeeze() * (images_full.shape[-1] - 1), 
                            int(star_encoder.tile_coords[indx, 0]), 
                            int(star_encoder.tile_coords[indx, 1]), 
                            subimage_slen = star_encoder.stamp_slen)

axarr[0].axvline(x=2, color = 'r')
axarr[0].axvline(x=6, color = 'r')
axarr[0].axhline(y=2, color = 'r')
axarr[0].axhline(y=6, color = 'r')


# On any arbitrary patch of the image 

In [ ]:
w = 10
x0_vec = np.arange(star_encoder.edge_padding, 
                   star_encoder.full_slen - star_encoder.edge_padding - w, 
                  w)

x1_vec = x0_vec

In [ ]:
x0_vec

In [ ]:
f, axarr = plt.subplots(1, 3, figsize=(16, 6))

x0 = int(np.random.choice(x0_vec, 1))
x1 = int(np.random.choice(x1_vec, 1))



plotting_utils.plot_subimage(axarr[0], images_full.squeeze(),
                            map_locs_full_image.squeeze() * (images_full.shape[-1] - 1), 
                            true_full_locs.squeeze() * (images_full.shape[-1] - 1), 
                            x0, x1, 
                            subimage_slen = w, 
                            add_colorbar = True, 
                            global_fig = fig)

axarr[0].set_title('observed; coords = {}'.format([x0, x1]));

plotting_utils.plot_subimage(axarr[1], vae_recon_mean.squeeze(),
                            map_locs_full_image.squeeze() * (images_full.shape[-1] - 1), 
                            None,  
                            x0, x1, 
                            subimage_slen = w, 
                            add_colorbar = True, 
                            global_fig = fig)

axarr[1].set_title('reconstructed; coords = {}'.format([x0, x1]));


residual = (vae_recon_mean.squeeze() - images_full.squeeze())
plotting_utils.plot_subimage(axarr[2], residual, 
                            map_locs_full_image.squeeze() * (images_full.shape[-1] - 1), 
                            None,  
                            x0, x1, 
                            subimage_slen = w, 
                            add_colorbar = True, 
                            global_fig = fig)

axarr[1].set_title('residual; coords = {}'.format([x0, x1]));


In [ ]:
torch.where((star_encoder.tile_coords[:, 0] == 32) & (star_encoder.tile_coords[:, 1] == 56))

In [ ]:
_psf = simulated_datasets_lib._trim_psf(simulated_dataset.simulator.psf, 
                                        star_encoder.stamp_slen - 2 * star_encoder.edge_padding)

In [ ]:
indx = np.arange(0, 20)
# indx = torch.where(torch.abs(map_locs - true_subimage_locs).sum(-1).sum(-1) > 0.5)[0]
# indx = true_n_stars == 2

plotting_utils.print_results(star_encoder, 
                                image_stamps[indx], 
                                _backgrounds[indx], 
                                _psf, 
                                true_subimage_locs[indx], 
                                is_on_array[indx],
                                use_true_n_stars = False)

In [ ]:
indx = np.arange(20, 40)
plotting_utils.print_results(star_encoder, 
                                image_stamps[indx], 
                                _backgrounds[indx], 
                                _psf, 
                                true_subimage_locs[indx], 
                                is_on_array[indx],
                                use_true_n_stars = False)

In [ ]:
# look at one image in detail

In [ ]:
indx = 35 # int(np.random.choice(image_stamps.shape[0], 1))

In [ ]:
image_i = image_stamps[indx].unsqueeze(0)
true_n_stars_i = true_n_stars[indx]
true_locs_i = true_subimage_locs[indx]

print('true n_stars: ', true_n_stars_i)

In [ ]:
n_stars_i = torch.Tensor([true_n_stars_i]).type(torch.LongTensor)

logit_loc_mean, logit_loc_log_var, \
    log_flux_mean, log_flux_log_var, log_probs = \
        star_encoder(image_i, data_params['sky_intensity'], n_stars_i)

In [ ]:
plotting_utils.plot_categorical_probs(log_probs[0], plt)

In [ ]:
_images = image_utils.trim_images(image_i, star_encoder.edge_padding)
plt.matshow(_images.squeeze())
plt.scatter(true_locs_i[is_on_array[indx], 1] * (star_encoder.stamp_slen - 1 - 2 * star_encoder.edge_padding), 
           true_locs_i[is_on_array[indx], 0] * (star_encoder.stamp_slen - 1 - 2 * star_encoder.edge_padding))

map_locs = torch.sigmoid(logit_loc_mean).detach()

plt.scatter(map_locs[0, 0:n_stars_i, 1] * (star_encoder.stamp_slen - 1 - 2 * star_encoder.edge_padding), 
           map_locs[0, 0:n_stars_i, 0] * (star_encoder.stamp_slen - 1 - 2 * star_encoder.edge_padding), 
           color = 'red', marker = 'x')

# check out deblending properties

In [ ]:
n_trials = 10

_n_stars = (torch.ones(n_trials) * 2).type(torch.LongTensor)
_fluxes = torch.ones(n_trials, max_stars) * simulated_dataset.f_min * 100

_locs = torch.rand(n_trials, max_stars, 2)

dist = 0.0
incr = 0.01
for i in range(_locs.shape[0]):
    dist = dist + incr
    _locs[i, 0, :] = 0.5 + dist
    _locs[i, 1,:] = 0.5 - dist

In [ ]:
_images = simulated_dataset.simulator.draw_image_from_params(_locs, _fluxes, _n_stars,
                                                             add_noise = False)

_backgrounds = torch.ones(10, 1, 1, 1) * simulated_dataset.sky_intensity

In [ ]:
plotting_utils.print_results(star_encoder, 
                            _images, 
                            _backgrounds, 
                            simulated_dataset.simulator.psf, 
                            _locs,
                            _n_stars, 
                            use_true_n_stars = False)

In [ ]:
import sdss_dataset_lib

In [ ]:
hubble_cat_file='../hubble_data/NCG7078/hlsp_acsggct_hst_acs-wfc_ngc7078_r.rdviq.cal.adj.zpt.txt'
sdss_hubble_data = sdss_dataset_lib.SDSSHubbleData(hubble_cat_file=hubble_cat_file, 
                                                   slen = 11, 
                                                   run = 2566, 
                                                   camcol = 6, 
                                                   field = 65, 
                                                max_detections = max_stars)

In [ ]:
len(sdss_hubble_data)

In [ ]:
# true parameters
hubble_loader = torch.utils.data.DataLoader(
                 dataset=sdss_hubble_data,
                 batch_size=len(sdss_hubble_data),
                 shuffle=False)

for _, data in enumerate(hubble_loader):
    hubble_fluxes = data['fluxes'].type(torch.float)
    hubble_locs = data['locs'].type(torch.float)
    hubble_n_stars = data['n_stars']
    sdss_images = data['image']
    sdss_backgrounds = data['background']
    
    break

In [ ]:
plt.hist(torch.log10(images - simulated_dataset.sky_intensity).flatten());

In [ ]:
plt.hist(torch.log10(sdss_images - sdss_backgrounds).flatten());

In [ ]:
indx = np.arange(0, 10)
plotting_utils.print_results(star_encoder, 
                                sdss_images[indx],
                                sdss_backgrounds[indx],
                                simulated_dataset.simulator.psf, 
                                hubble_locs[indx],
                                hubble_n_stars[indx], 
                                use_true_n_stars = False, 
                                residual_clamp = 1e16)